In [50]:
import os
import glob
import pandas as pd
import numpy as np
import collections

print(os.getcwd())

c:\Users\Elias\Desktop\GitHub\d-hub\data\senaem\testing


In [3]:
df = pd.read_csv('../downloads/senae_importacion_simplificado_enero_diciembre_2013.csv')

In [ ]:
titus = pd.read_excel('../files_support/chapters_only.xlsx')
titus['TÍTULO'] = titus['TÍTULO'].str.split(pat = '.', expand=True)[1]
titus.set_index('CAPÍTULO', inplace=True)
sa = titus.to_dict(orient='dict')['TÍTULO']

In [117]:
anio_list = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
for anio in anio_list:
    file_paths_anio = glob.glob(os.path.join('../downloads/', f'senae_importacion*{anio}.csv'))
    
    print(file_paths_anio)

['../downloads\\senae_importacion_general_enero_junio_2013.csv', '../downloads\\senae_importacion_general_julio_diciembre_2013.csv', '../downloads\\senae_importacion_simplificado_enero_diciembre_2013.csv']
['../downloads\\senae_importacion_general_enero_junio_2014.csv', '../downloads\\senae_importacion_general_julio_diciembre_2014.csv', '../downloads\\senae_importacion_simplificado_enero_diciembre_2014.csv']
['../downloads\\senae_importacion_general_enero_junio_2015.csv', '../downloads\\senae_importacion_julio_diciembre_2015.csv', '../downloads\\senae_importacion_simplificado_enero_diciembre_2015.csv']
['../downloads\\senae_importacion_general_enero_junio_2016.csv', '../downloads\\senae_importacion_julio_diciembre_2016.csv', '../downloads\\senae_importacion_simplificado_enero_diciembre_2016.csv']
['../downloads\\senae_importacion_general_enero_junio_2017.csv', '../downloads\\senae_importacion_general_julio_diciembre_2017.csv', '../downloads\\senae_importacion_simplificado_enero_diciemb

In [178]:
varnames = [
    'TIPO_IMPORTACION', 'FEC_INGRESO', 'DISTRITO', 'SUBPARTIDA', 
    'PAIS_ORIGEN', 'FOB',  'FLETE', 'SEGURO', 'CIF', 'SALVAGUARDIA', 
    'SALVAGUARDIA_ESPECIFICA', 'FODINFA', 'ICE_ADVALOREM', 
    'ICE_ESPECIFICO', 'IVA', 'ADVALOREM', 'ADVALOREM_ESPECIFICO'
]

file_paths_2017 = ['../downloads\\senae_importacion_general_enero_junio_2017.csv', '../downloads\\senae_importacion_general_julio_diciembre_2017.csv', '../downloads\\senae_importacion_simplificado_enero_diciembre_2017.csv']

dfs = []

for file_path in file_paths_2017:
    df_temp = pd.read_csv(file_path, usecols=varnames, nrows=100000)
    dfs.append(df_temp)

df = pd.concat(dfs, ignore_index=True)

In [114]:
# Note: The dataframe for simplified imports has 2 columns less than 
pd.options.display.max_columns = 50
dfs[0].head()



,TIPO_IMPORTACION,FEC_INGRESO,DISTRITO,SUBPARTIDA,PAIS_ORIGEN,FOB,FLETE,SEGURO,CIF,SALVAGUARDIA,SALVAGUARDIA_ESPECIFICA,FODINFA,ICE_ADVALOREM,ICE_ESPECIFICO,IVA,ADVALOREM,ADVALOREM_ESPECIFICO
0,IMP.GRAL.,2017-02-24,019-GUAYAQUIL - AEREO,9405920000,FI-FINLANDIA,2368.20,109.50,24.78,2502.48,875.87,0.0,12.52,0.0,0.0,562.31,625.63,0.0
1,IMP.GRAL.,2017-05-22,019-GUAYAQUIL - AEREO,3917400000,US-ESTADOS UNIDOS,163.75,18.76,0.89,183.40,0.00,0.0,0.94,0.0,0.0,30.40,28.19,0.0
2,IMP.GRAL.,2017-06-22,019-GUAYAQUIL - AEREO,4009410000,JP-JAPON,279.88,14.52,0.22,294.62,0.00,0.0,1.47,0.0,0.0,40.84,44.19,0.0
3,IMP.GRAL.,2017-01-04,019-GUAYAQUIL - AEREO,8413819000,SE-SUECIA,25615.30,0.00,43.61,25658.91,0.00,0.0,140.89,0.0,0.0,4162.07,1408.96,0.0
4,IMP.GRAL.,2017-04-10,019-GUAYAQUIL - AEREO,4009410000,US-ESTADOS UNIDOS,25.75,0.86,0.02,26.63,0.00,0.0,0.13,0.0,0.0,4.31,3.99,0.0


In [60]:
columns = []
for df in dfs:
    columns.extend(df.columns.tolist())
# Interesting, only `TIPO_UNIDAD_COMERCIAL` and 'CANTIDAD_COMERCIAL' are lacking\
# in the simplified regime dataset. I am now unlike to use them so I'll\
# throw them away
[(item, count) for item, count in collections.Counter(columns).items() if count ==2]



[]

I now must define which variables I want to see in the final files.

Dimensionality of the importacion_simplificado frames is that number of rows keeps growing steadily It goes from around 100'000 to over 250'000. Important.\
They all seem to have the same columns which means I may safely procceed.


In [179]:
# create `date` datetime column from FEC_INGRESO
df['date'] = pd.to_datetime(df['FEC_INGRESO'])

# Separate the variable 'PAIS_DESTINO' into iso code and country name variables
paisD_expanded = df['PAIS_ORIGEN'].str.split(pat='-', expand=True)
df['iso'] = paisD_expanded[0]
df['pais'] = paisD_expanded[1]

# GET CODES FOR CHAPTERS IN THE GENERAL CLASSIFICATION OF PRODUCTS: i) Change the 'SUBPARTIDA' column data type\
# to string. ii) create a new variable that tells us the length of the elements in 'SUBPARTIDA'. iii) \ Whenever
# 'SUBPARTIDA' is of length 9, add a 0 at the beginning. 'SUBPARTIDA' must be 10-digit. iv) The chapter reference \
# are the first two characters/digits in 'SUBPARTIDA'
df['SUBPARTIDA'] = df['SUBPARTIDA'].astype('str')
df['SUBPART_LENGTH'] = df['SUBPARTIDA'].str.len()
df['SUBPARTIDA'] = np.where(df['SUBPART_LENGTH'] == 9, '0' + df['SUBPARTIDA'], df['SUBPARTIDA'])
df['chap'] = df['SUBPARTIDA'].str.slice(stop=2)

# Add up all duties to form a single column: `import_duties`
duties=[
    'SALVAGUARDIA', 
    'SALVAGUARDIA_ESPECIFICA', 
    'FODINFA', 
    'ICE_ADVALOREM', 
    'ICE_ESPECIFICO', 
    'IVA', 
    'ADVALOREM', 
    'ADVALOREM_ESPECIFICO'
]
# new var is going to be name derechos, as a rough translation of import duties
df['duties'] = df[duties].sum(axis=1)

# from variable `DISTRITO` extract specific code for the district as well as name
df['distr_code'] = df['DISTRITO'].str.replace(' ', '').str.split('-', expand=True)[0]
df['distr_city'] = df['DISTRITO'].str.replace(' ', '').str.split('-', expand=True)[1]
df['distr_compl'] = df['DISTRITO'].str.replace(' ', '').str.split('-', expand=True)[2]

df['distr_name'] = np.where(df['distr_compl'].isna(), df['distr_city'], df['distr_city'] + '-' + df['distr_compl'])



In [183]:
df.loc[df['distr_compl'] == 'MACARA', 'distr_city'] = 

8365      LOJA
11714     LOJA
14984     LOJA
28438     LOJA
32747     LOJA
39472     LOJA
47059     LOJA
54991     LOJA
63690     LOJA
79246     LOJA
91310     LOJA
94573     LOJA
106716    LOJA
110421    LOJA
116054    LOJA
116346    LOJA
119555    LOJA
120304    LOJA
124765    LOJA
126286    LOJA
132386    LOJA
134249    LOJA
137627    LOJA
141064    LOJA
157356    LOJA
181183    LOJA
187553    LOJA
Name: distr_city, dtype: object